<a href="https://colab.research.google.com/github/fernandofsilva/LSTM_Option_Pricing/blob/main/notebooks/03_modelo_dnn_sequential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
#@title Carregando as bibliotecas base
import pandas as pd
import numpy as np
import tensorflow as tf

import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn

seaborn.set_style('whitegrid')

In [2]:
#@title Carregando os dados
data = pd.read_csv(f'/content/drive/My Drive/Mestrado/data/dados_treino_teste.csv.gz', compression='gzip', index_col=0)
data.head()

,codigo,mercado,preco_opcao,preco_exercicio,data_vencimento,T,preco_ativo,volatilidade,taxa_juros,black_scholes,delta_black_scholes,base
2015-01-05,PETRM17,OPÇÕES DE VENDA,8.18,16.91,2015-01-19,0.039683,8.61,0.771953,0.1157,8.22,100,treino
2015-01-05,PETRM28,OPÇÕES DE VENDA,0.36,8.41,2015-01-19,0.039683,8.61,0.771953,0.1157,0.41,40,treino
2015-01-05,PETRM2,OPÇÕES DE VENDA,0.02,4.91,2015-01-19,0.039683,8.61,0.771953,0.1157,0.00,0,teste
2015-01-05,PETRM23,OPÇÕES DE VENDA,0.79,9.21,2015-01-19,0.039683,8.61,0.771953,0.1157,0.87,63,treino
2015-01-05,PETRM25,OPÇÕES DE VENDA,1.13,9.61,2015-01-19,0.039683,8.61,0.771953,0.1157,1.16,73,treino


# Engenharia das variáveis (Feature Engineering)

Essa sessão é composta da transformação dos dados para entrada na rede na rede neural. Portando, as variáveis são transformadas do seu valor original, seja para adequação dentro da rede neural ou para um melhor treinamento da rede, essas transformações são:

- Variavéis númericas: preco_exercicio, preco_ativo, foram normalizadas antes da entrada na rede
- Variavéis númericas: preco_opcao (alvo), volatilidade, taxa_juros e T não sofreram alterações
- Variável categórica mercado sofreu one hot encoding

A transformação dos dados é feita no mesmo momento que o modelo é treinado, isso é feito através de uma camada dentro do modelo, essa camada tem o nome de feature layer.

In [3]:
#@title Pipeline de entrada dos dados
def df_to_dataset(dataframe, base, shuffle=True, batch_size=22):

    # Criar cópia do dataframe
    dataframe = dataframe.copy()

    # Filtrar a base
    dataframe = dataframe[dataframe['base'] == base]

    # Variavel alvo
    labels = dataframe.pop('preco_opcao')

    # Colunas do modelo
    cols = ['mercado', 'preco_exercicio', 'preco_ativo', 'T', 'volatilidade', 'taxa_juros']

    # Criar o td.data
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe[cols]), labels))

    # Embaralhar os dados se necessário
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))

    # Criar o batch de dados
    ds = ds.batch(batch_size)

    return ds

# Divisão da base de treino e teste
train_ds = df_to_dataset(data, base='treino')
test_ds = df_to_dataset(data, shuffle=False, base='teste')

In [4]:
#@title Mapeamento das colunas
feature_columns = []

# Colunas númericas normalizadas
for column in ['preco_exercicio', 'preco_ativo']:
    
    mean = data.loc[data['base'] == 'treino', column].mean()
    stdev = data.loc[data['base'] == 'treino', column].std()

    feature_columns.append(tf.feature_column.numeric_column(column, normalizer_fn = lambda x: (x - mean) / stdev))

# Colunas númericas sem normalização
for column in ['T', 'volatilidade', 'taxa_juros']:

    feature_columns.append(tf.feature_column.numeric_column(column))

# Colunas categóricas
option = tf.feature_column.categorical_column_with_vocabulary_list('mercado', ['OPÇÕES DE COMPRA', 'OPÇÕES DE VENDA'])
option_one_hot = tf.feature_column.indicator_column(option)
feature_columns.append(option_one_hot)

In [5]:
#@title Camada de transformação (feature layer)
feature_layer = tf.keras.layers.DenseFeatures(feature_columns, name='Feature')

# Modelo

O modelo de rede neural profunda a seguir, foi baseado nos estudos desenvolvidos por Hirsa, Karatas, & Oskoui. No trabalho são testadas diversas arquiteturas (camadas e elementos em cada camada), bem como função de ativão de cada camada e também função de otimização.

A conclusão do estudo mostra que os melhores resultados foram obtidos utilizando uma rede de 4 camadas com 120 neurônios cada uma.

In [15]:
#@title Criar, compilar e treinar o modelo
# Define Callback
class mse_Callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_loss')<0.075):
      print("\nReached Mean Squared Erro less than 0.075, it is cancelling training!")
      self.model.stop_training = True

# Define de model
model = tf.keras.Sequential([
  feature_layer,
  tf.keras.layers.Dense(120, activation='relu'),
  tf.keras.layers.Dense(120, activation='relu'),
  tf.keras.layers.Dense(120, activation='relu'),
  tf.keras.layers.Dense(120, activation='elu'),
  tf.keras.layers.Dense(1)
])

# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='mse',
    metrics=[tf.keras.metrics.MeanAbsoluteError(name="MAE", dtype=None)]
)

# Instantiete callbacks
callbacks = mse_Callback()

# Train the model
history = model.fit(train_ds, validation_data=test_ds, epochs=200, callbacks=[callbacks])

Epoch 1/200
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
7402/7416 [============================>.] - ETA: 0s - loss: 0.1094 - MAE: 0.1812WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'mercado': <tf.Tensor 'ExpandDims_1:0' shape=(None, 1) dtype=string>, 'preco_exercicio': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=float64>, 'preco_ativo': <tf.Tensor 'ExpandDims_2:0' shape=(None, 1) dtype=float64>, 'T': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=float64>, 'volatilidade': <tf.Tensor 'ExpandDims_5:0' shape=(None, 1) dtype=float64>, 'taxa_juros': <tf.Tensor 'ExpandDims_4:0' shape=(None, 1) dtype=float64>}
Consider rewriting this model with the Functional API.

Reached Mean Squared Erro less than 0.075, it is cancelling training!
7416/7416 [==============================] - 17s 2ms/step - loss: 0.1093 - MAE: 0.1812 - val_loss: 0.0704 

In [ ]:
#@title Plot das Métricas do Modelo
# Split variables
train_loss = history.history['loss']
test_loss = history.history['val_loss']
train_mae = history.history['MAE']
test_mae = history.history['val_MAE']
train_mape = history.history['MAPE']
test_mape = history.history['val_MAPE']
epoch = range(len(test_mape))

# Create two subplots
fig, axs = plt.subplots(3, 1, figsize=(20, 8))
axs[0].plot(epoch, train_loss, 'tab:blue', label='treino')
axs[0].plot(epoch, test_loss, 'tab:red', label='teste')
axs[0].set_title('Função de Perda')
axs[0].set(ylabel='Perda')
# axs[0].set_ylim([0, 1])
axs[0].legend()

axs[1].plot(epoch, train_mae, 'tab:blue', label='treino')
axs[1].plot(epoch, test_mae, 'tab:red', label='teste')
axs[1].set_title('Erro Médio Absoluto')
axs[1].set(ylabel='MAE')
# axs[1].set_ylim([0, 2])
axs[1].legend()

axs[2].plot(epoch, train_mape, 'tab:blue', label='treino')
axs[2].plot(epoch, test_mape, 'tab:red', label='teste')
axs[2].set_title('Erro Médio Percentual Absoluto')
axs[2].set(ylabel='MAPE')
# axs[2].set_ylim([0, 2])
axs[2].legend()

for ax in axs.flat:
    ax.set(xlabel='Épocas')

for ax in axs.flat:
    ax.label_outer()

plt.show()

In [ ]:
# Gráficos das opções

Para comparação foram escolhidas aleatóriamente 10 opções da base de teste.

In [ ]:
#@title Função de comparação
from sklearn.metrics import mean_squared_error

def predict(model, data, options, expire):

    # Select options
    df = data[(data['option'] == options) & (data['expire'] == expire)]

    # Convert to dataset
    ds = df_to_dataset(df, shuffle=False)

    # Predict values
    pred = model.predict(ds)

    rmse_bs = np.sqrt(mean_squared_error(df['value'],df['bs']))
    rmse_dnn = np.sqrt(mean_squared_error(df['value'], pred))
    if rmse_dnn < rmse_bs:
        result = 'Rede Neural'
    else:
        result = 'Black-Scholes'

    fig, axs = plt.subplots(figsize=(20, 4))
    axs.plot(df.index, pred, 'tab:blue', label='DNN')
    axs.plot(df.index, df['bs'], 'tab:green', label='BS')
    axs.plot(df.index, df['value'], 'tab:red', label=options)
    axs.set_title(f'{options} - Black-Scholes:{rmse_bs:.4f}, DNN:{rmse_dnn:.4f}, Melhor modelo {result}')
    axs.set(ylabel='Valor R$')
    axs.set(xlabel='data')
    axs.legend()

In [ ]:
#@title 
for option_type in ['call', 'put']:
    
    options = test.loc[test['option_type'] == option_type, ['option', 'expire']].drop_duplicates()

    for _ in range(5):

        rand = np.random.randint(options.shape[0])

        predict(
            model,
            data,
            options.iloc[rand, 0],
            options.iloc[rand, 1]
        )

# Reference

Hirsa, A., Karatas, T., & Oskoui, A. (2019). Supervised deep neural networks (DNNS) for pricing/calibration of vanilla/exotic options under various different processes.